In [ ]:
from subtitle_filter import Subtitles
from difflib import SequenceMatcher
from collections import namedtuple
from pysubparser import parser
from itertools import groupby
from hazm import *
import os, os.path
import pathlib
import shutil
import pysrt
import re

In [ ]:
#clear subtitle texts and make it UFT-8
hazfiat=[]
def clearance(path):
    normalizer = Normalizer()
    try:
        sub = pysrt.open(path)
        a = True
    except:
        file_to_rem = pathlib.Path(path)
        a = False
    if a == True:
        for i in range (len(sub)):
            su = sub[i].text
            s = normalizer.normalize(su)
            clean = re.findall(r'[.ا.آ.ب.پ.ت.ث.ج.چ.ح.خ.د.ذ.ر.ز.ژ.س.ش.ص.ض.ط.ظ.ع.غ.ف.ق.ک.گ.ل.م.ن.و.ه.ی.ي.ئ.u./.\d.،.۱.۲.۳.۴.۵.۶.۷.۸.۹.۰.(.).؟.?]+' , s)
            SubtitleText = ' '.join (clean)
            del sub[i].text
            sub[i].text = s #SubtitleText
        sub.save(path, encoding='utf-8')
    if a == False:
        return file_to_rem

In [ ]:
#adjust dirctories
def directory(ls,parent_dir):
    if ls[1]:
        files1 = next(os.walk(ls[0] +'/'+ ls[1][0]))
        if not files1[1]:
            for fils in files1[2]:
                shutil.move(files1[0] +'/'+ fils,ls[0])
#             os.chdir(parent_dir)
#             shutil.rmtree(files1[0])
        else:
            for fls in files1[1]:
                fls1 = next(os.walk(files1[0] +'/'+ fls))
                if not fls1[1]:
                    for fils in fls1[2]:
                        shutil.move(fls1[0] +'/'+ fils,ls[0])
                    os.chdir(parent_dir)
                    shutil.rmtree(fls1[0])
                else:
                    for f in fls1[1]:
                        fi = next(os.walk(fls1[0] +'/'+ f))
                        for fils in fi[2]:
                            shutil.move(fi[0] +'/'+ fils,ls[0])
                        os.chdir(parent_dir)
                        shutil.rmtree(fi[0])
    
    if len(ls[2]) < 2 :
        os.chdir(parent_dir)
        shutil.rmtree(ls[0])

In [ ]:
#A way to parse subtitle
def subtitles(path):
    subs=[]
    with open(path) as f:
        res = [list(g) for b,g in groupby(f, lambda x: bool(x.strip())) if b]
        Subtitle = namedtuple('Subtitle', 'number start end content')
        for sub in res:
            if len(sub) >= 3: # not strictly necessary, but better safe than sorry
                sub = [x.strip() for x in sub]
                number, start_end, *content = sub # py3 syntax
                start, end = start_end.split(' --> ')
                subs.append((number, start, end, content))
    return subs

In [ ]:
#Another way to parse 
def subtitles1 (path):
    subs=[]
    sub = parser.parse(path)
    for su in sub:
        clean = re.findall(r'[ئابپتثجچحخآدذرزسشصضطظعغفقکگلمنوهی]+' , str(su))
        SubtitleText = ''.join (clean)
        subs.append(SubtitleText) 
    return subs

In [ ]:
#make subtitle ready to compare
def parse(subs):
    sub =[]
    f = []
    for i in range(0,len(subs)):
        clean = re.findall(r'[ ئابپتثجچحخدآذرزسشصضطظعغفقکگلمنوهی  ]+' , subs[i])
        SubtitleText = ''.join (clean)
        if SubtitleText :
            sub.append(SubtitleText)
        f = ''.join(sub)
    return f

In [ ]:
# remove when a subtitle hasn't appropriate Codec
def persian(subs,f,path):
    try:
        if len(subs)*5 > len(f) or len(subs)== 0 :
            try:
                os.remove(path)
            except:
                nshod.append(path)
    except:
        os.remove(path)

In [ ]:
#check out similarity
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
#Remove a file from a directory
def remove(path):
    file_to_rem = pathlib.Path(path)
    file_to_rem.unlink()

In [ ]:
#addressing and findout locations in memory
parent_dir = "path to subtitle's folder"
files = next(os.walk(parent_dir))
len(files[1])

In [ ]:
#main body of the code for removing similar subtitles
delete = []
for file in files[1]:
    print (r'{}'.format(file))
    if delete:
        for delet in delete:
            try:
                remove(delet)
            except:
                delete.remove(delet)
                continue
    os.chdir(parent_dir)
    path = os.path.join(parent_dir, file)
    fils = next(os.walk(path))
    fis = len(fils[2])
    for i in range (0,fis-1):
        k = i+1
        path1 = str (fils[0] +'/'+ fils[2][i])
        try:
            first = subtitles1(path1)
        except:
            delete.append(path1)
            continue
        for j in range (k,fis):
            path2 = str (fils[0] +'/'+ fils[2][j])
            try:
                second = subtitles1(path2)
            except:
                delete.append(path2)
                continue
            if similar(first,second)> 0.95 :
                delete.append(path2)
                print(similar(first,second))

In [ ]:
#clear the text of the subtitles
hazf=[]
for file in files[1]:
    print (r'{}'.format(file))
    os.chdir(parent_dir)
    path = os.path.join(parent_dir, file)
    fils = next(os.walk(path))
    for fis in fils[2]:
        hazf.append(clearance(fils[0] +'/'+ fis))
for haz in hazf:
    remove(haz)

In [ ]:
#remove directories with one or less subtitle
parent_dir = "path to folder of subtitles"
parentt_dir = parent_dir
files = next(os.walk(parent_dir))
len(files[1])
for file in files[1]:
    print (r'{}'.format(file))
    os.chdir(parent_dir)
    path = parent_dir+file
    fils = next(os.walk(path))
    try:
        directory(fils,parentt_dir)
    except:
        continue

In [ ]:
#Note: in this section first, you shoud download "Subtitle Speech Synchronizer" 
#then you should select batch input
#here we produced this code to creat Yaml list to initial the batch processing 
#it gets two subtitles from each film and makes them syncronizing by each other 
#attention to number of your subtitle input 
#if you have more than 250 compares get the list into several splits by 100 compares then initial each batches seprately
#enjoy!

#creat yaml file for sycnronizing
yamllist = []
for file in files[1]:
    os.chdir("/Users/Jama/filmsSRT")
    path = os.path.join(parent_dir, file)
    fils = next(os.walk(path))
    fis = len(fils[2])
    for i in range (0,fis-1):
        k = i+1
        path1 = str (fils[0] +'/'+ fils[2][i])
        path1 = path1.replace('/',"\\")
        for j in range (k,fis):
            path2 = str (fils[0] +'/'+ fils[2][j])
            path2 = path2.replace('/','\\')
            out= "- out:\n    enc: UTF-8\n    path: '{ref_dir}\{ref_name}{if:sub_lang:.}{sub_lang}.srt'"
            ref= '  ref:\n    lang: per\n    path: C:'
            stream= '    stream: 1'
            sub= '  sub:\n    lang: per\n    path: C:'
            base = '{}\n{}{}\n{}\n{}{}\n{}\n'
            yaml = base.format(out,ref,path1,stream,sub,path2,stream)
            yamllist.append(yaml)
with open('listyaml.txt', 'w',encoding="utf-8") as f:
    f.writelines(yamllist)         


In [ ]:
#last, you need to cahnge directory
os.chdir('/Users')